In [13]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt


In [6]:
df = pd.read_csv('E:\Practice\ML Practice\ML lab\Lab5\heart.csv')
X = df[['restecg', 'oldpeak']]
y = df['target']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [7]:
dt = DecisionTreeClassifier(random_state=42)
knn = KNeighborsClassifier()
rf = RandomForestClassifier(random_state=42)
xgb = XGBClassifier(eval_metric='mlogloss', random_state=42)

In [9]:
voting_hard = VotingClassifier(
    estimators=[('dt', dt), ('rf',rf), ('xgb', xgb)],
    voting='hard'
)

voting_hard.fit(X_train_scaled, y_train)
y_pred_hard = voting_hard.predict(X_test_scaled)
print("Hard Voting Accuracy:", accuracy_score(y_test, y_pred_hard))

Hard Voting Accuracy: 0.6878048780487804


In [10]:
voting_soft = VotingClassifier(
    estimators=[('dt', dt), ('knn', knn), ('rf', rf), ('xgb', xgb)],
    voting='soft'
)
voting_soft.fit(X_train_scaled, y_train)
y_pred_soft = voting_soft.predict(X_test_scaled)
print("Soft Voting Accuracy:", accuracy_score(y_test, y_pred_soft))

Soft Voting Accuracy: 0.6878048780487804


In [11]:
from itertools import product
best_weights = None
best_acc = 0

for weight in product([1, 2, 3], repeat=4):
    weighted = VotingClassifier(
        estimators=[("dt", dt), ("knn", knn), ("rf", rf), ("xgb", xgb)],
        voting="soft",
        weights=weight
    )
    weighted.fit(X_train, y_train)
    soft_acc = accuracy_score(y_test, weighted.predict(X_test))

    if soft_acc > best_acc:
        best_acc = soft_acc
        best_weight = weight

print(f"Best Weights: {best_weight}, Best Accuracy: {best_acc}")

Best Weights: (1, 1, 1, 1), Best Accuracy: 0.6878048780487804
